# Create and Test Scoring Pipeline and Deploy R Shiny Dashboard App

### Introduction

Now that we have built the machine learning model, stored and deployed it using [ibm-watson-machine-learning](http://ibm-wml-api-pyclient.mybluemix.net), we can use the model to score new data. 

In this notebook we will:

* Programmatically get the ID's for the deployment space and model deployment that were created in the **`2-model-training`** notebook.
* Promote assets required for scoring new data into the deployment space.
* Create a deployable function which will take raw data for scoring, prep it into the format required for the model and score it.
* Deploy the function.
* Create the required payload, invoke the deployed function and return predictions. <br>

In the second part we will:

* Store Shiny assets into the same deployment space.
* Deploy Shiny assets as an app and view the dashboard.

In [15]:
import pandas as pd
import datetime
from ibm_watson_machine_learning import APIClient
import os
import string
import random

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "4.6"
}

client = APIClient(wml_credentials)

### User Inputs

Enter the path to the csv file with raw data to be scored.

In [16]:
# specify the location of the csv file with raw customer data that we would like to score for
dataset_loc = '/project_data/data_asset/customer_history.csv'
dataset_name = os.path.basename(dataset_loc)

### Set up Deployment Space, Deployments and Assets

The following code programmatically gets the deployment space and the model deployment details which were created in **2-model-training**. 
We use the space name and deployment names that were used when creating the deployments as specified below. 
If multiple deployments within the selected space have the same name, the most recently created deployment is used. 

Alternatively, the user can manually enter the space and deployment id's.

The code also promotes some assets into the deployment space, specifically, the dataset with raw data for scoring, the python script file which is used for prepping the data and the metadata that was stored when prepping the data. By promoting these assets into the deployment space, they are available and can be accessed by the deployed function. 

In [17]:
space_name = 'Customer Attrition Space'
model_name = 'Customer Attrition Prediction Model'
deployment_name = 'Customer-Attrition-Prediction-Model-Deployment'

Get the space we are working in, which is found using the name that were hardcoded in **2-model-training**. 
If the user would like to use a different space manually set the **space_id**.

Set the space as the default space for working.

In [18]:
l_space_details = []
l_space_details_created_times = []
for space_details in client.spaces.get_details()['resources']:
    if space_details['entity']['name'] == space_name:
        space_id=space_details['metadata']['id']

# set this space as default space
client.set.default_space(space_id)

'SUCCESS'

Get the deployment id. If there are multiple deployments with the same name in the same space, we take the latest.

In [19]:
l_deployment_details = []
l_deployment_details_created_times = []

for deployment in client.deployments.get_details()['resources']:
        

        if deployment['entity']['name'] == deployment_name:            
                l_deployment_details.append(deployment)
                l_deployment_details_created_times.append(datetime.datetime.strptime(deployment['metadata']['created_at'],  '%Y-%m-%dT%H:%M:%S.%fZ'))
                

# get the index of the latest created date from the list and use that to get the deployment_id
list_latest_index = l_deployment_details_created_times.index(max(l_deployment_details_created_times))
deployment_id = l_deployment_details[list_latest_index]['metadata']['id']

### Promote Assets to Deployment space

Promote the assets into the deployment space. We will use the prep script for getting the raw data into the format required for scoring. We also need the prep metadata that was saved as json during the prep for training, this ensures that the user inputs specified for prepping the data for training are the same used for scoring. We add these assets into the deployment space.  Also store the raw data dataset in the deployment space.

In [20]:
# we will use the prep script for getting the raw data into the format required for scoring
# we also need the prep metadata that was saved as json during the prep for training - this ensures that the user inputs specified for prepping the data for training are the same used for scoring
# we need to add these files into the deployment space

asset_details_json = client.data_assets.create('training_user_inputs_and_prepped_column_names.json', file_path='/project_data/data_asset/training_user_inputs_and_prepped_column_names.json')
asset_details_script = client.data_assets.create('attrition_prep.py', file_path='/project_data/data_asset/attrition_prep.py')

asset_details_dataset = client.data_assets.create(dataset_name, file_path=dataset_loc)

Creating data asset...
SUCCESS
Creating data asset...
SUCCESS
Creating data asset...
SUCCESS


In [21]:
client.data_assets.list()

--------------------------------------------------  ----------  --------  ------------------------------------
NAME                                                ASSET_TYPE  SIZE      ASSET_ID
customer_history.csv                                data_asset  22543588  1f43095c-c80f-4e6b-a411-f4b1c7250c2b
training_user_inputs_and_prepped_column_names.json  data_asset  4337      e1425c18-48ca-480b-89b6-b970d8b6d1e9
customer_history.csv                                data_asset  22543588  7fdcc01e-7944-4f71-932c-43576da98734
training_user_inputs_and_prepped_column_names.json  data_asset  4337      c98a5186-b4cb-43fd-89c2-83d6d3bcc3b5
attrition_prep.py                                   data_asset  36102     5530c67c-a36f-4379-b7a8-e59ed65ec07c
customer_history.csv                                data_asset  22543588  9659f1f9-7955-4e00-8ebe-4df5d9886e8f
attrition_prep.py                                   data_asset  36102     732ef294-6f06-488a-9784-4f67b21c0df6
customer_history.csv         

## Create the Deployable Function

Functions can be deployed in Watson Machine Learning in the same way models can be deployed. The python client or REST API can be used to send data to the deployed function. Using the deployed function allows us to prepare the data and pass it to the model for scoring all within the deployed function.

We start off by creating the dictionary of default parameters to be passed to the function. We get the ID's of all assets that have been promoted into the deployment space. We also add the model deployment ID and space ID information into the dictionary.

In [22]:
# get the assets that were stored in the space - in this version of the package we need to manually assign the id
metadata_id = asset_details_json['metadata']['guid']
prep_id = asset_details_script['metadata']['guid']
dataset_id = asset_details_dataset['metadata']['guid']

In [23]:
assets_dict = {'dataset_asset_id' : dataset_id, 'metadata_asset_id' : metadata_id, 
                   'prep_script_asset_id' : prep_id, 'dataset_name' : dataset_name}

In [24]:
wml_credentials["instance_id"] = "openshift"

ai_parms = {'wml_credentials' : wml_credentials,'space_id' : space_id, 'assets' : assets_dict, 'model_deployment_id' : deployment_id}

### Scoring Pipeline Function

The function below takes new customers to be scored as a payload. It preps the customer raw data, loads the model, executes the model scoring and generates the predictions for attrition. 

The following rules are required to make a valid deployable function:

* The deployable function must include a nested function named "score".
* The score function accepts a list.
* The list must include an array with the name "values".
* The score function must return an array with the name "predictions", with a list as the value, which in turn contains an array with the name "values". Example: ```{"predictions" : [{'values' : }]}```
* We pass default parameters into the function, credentials and space detail, details of the assets that were promoted into the space and also the model deployment guid. 
* The assets are downloaded into the deployment space and imported as variables. The raw data to be scored is then prepared and the function calls the model deployment endpoint to score and return predictions. 

In [25]:
def scoring_pipeline(parms=ai_parms):
    
    import pandas as pd
    import requests
    import os
    import json
    
    from ibm_watson_machine_learning import APIClient
    client = APIClient(parms["wml_credentials"])
    client.set.default_space(parms['space_id'])
     
    
    # call the function to download the stored dataset asset and return the path
    dataset_path = client.data_assets.download(parms['assets']['dataset_asset_id'], parms['assets']['dataset_name'])
    df_raw = pd.read_csv(dataset_path, infer_datetime_format=True, 
                             parse_dates=['CUSTOMER_RELATIONSHIP_START_DATE', 
                                              'CUSTOMER_SUMMARY_END_DATE','CUSTOMER_SUMMARY_START_DATE'])

    # call the function to download the prep script and return the path
    prep_script_path = client.data_assets.download(parms['assets']['prep_script_asset_id'], 'prep_data_script.py')
    # remove the rest of path and .py at end of file name to get the name of the script for importing
    script_name = os.path.basename(prep_script_path).replace('.py', '')
    
    # call the function to download the prep metadata and return the path
    metadata_path = client.data_assets.download(parms['assets']['metadata_asset_id'], 'user_inputs.json')
    
    def prep(cust_id, sc_end_date):
        
        import requests
        import os
        # import the prep script that we downloaded into the deployment space
        prep_data_script = __import__(script_name)
        
        with open(metadata_path, 'r') as f:
            user_inputs_dict = json.load(f)
        
        globals().update(user_inputs_dict)
                  
        input_df = df_raw[df_raw[granularity_key] == cust_id]
        
        scoring_prep = prep_data_script.AttritionPrep('score', effective_date=sc_end_date, feature_attributes=feature_attributes,
                             derive_column_list=derive_column_list,
                             granularity_key=granularity_key, target_attribute=target_attribute,
                             status_attribute=status_attribute,
                             funds_attribute=funds_attribute, date_customer_joined=date_customer_joined,
                             customer_end_date=customer_end_date, customer_start_date=customer_start_date,
                             period_attribute=period_attribute, status_flag_attrition=status_flag_attrition,
                             AUM_reduction_threshold=AUM_reduction_threshold,
                             forecast_horizon=forecast_horizon, observation_window=observation_window,
                             sum_list=sum_list, cat_threshold=cat_threshold)
        
        prepped_data = scoring_prep.prep_data(input_df, 'score')
        
        if prepped_data is None:
            print("Data prep filtered out customer data. Unable to score.", file=sys.stderr)
            return None
    
        # handle empty data
        if prepped_data.shape[0] == 0:
            print("Data prep filtered out customer data. Unable to score.", file=sys.stderr)
            return None

        to_drop_corr = ['CUSTOMER_SUMMARY_FUNDS_UNDER_MANAGEMENT_mean', 'CUSTOMER_SUMMARY_FUNDS_UNDER_MANAGEMENT_min',
                            'CUSTOMER_SUMMARY_FUNDS_UNDER_MANAGEMENT_max', 'CUSTOMER_SUMMARY_TOTAL_AMOUNT_OF_DEPOSITS_min',
                            'CUSTOMER_SUMMARY_TOTAL_AMOUNT_OF_DEPOSITS_max', 'CUSTOMER_SUMMARY_TOTAL_AMOUNT_OF_DEPOSITS_sum',
                            'CUSTOMER_ANNUAL_INCOME', 'CUSTOMER_NUMBER_OF_DEPENDENT_CHILDREN',
                            'NUM_ACCOUNTS_WITH_RISK_TOLERANCE_MODERATE', 'NUM_ACCOUNTS_WITH_RISK_TOLERANCE_HIGH',
                            'NUM_ACCOUNTS_WITH_RISK_TOLERANCE_VERY_LOW', 'NUM_ACCOUNTS_WITH_RISK_TOLERANCE_LOW', 
                            'CUSTOMER_TENURE', 'NUM_ACCOUNTS_WITH_INVESTMENT_OBJECTIVE_PLANNING',
                            'NUM_ACCOUNTS_WITH_INVESTMENT_OBJECTIVE_SECURITY','CUSTOMER_SUMMARY_TOTAL_AMOUNT_OF_DEPOSITS_max_min_ratio',
                            'CUSTOMER_SUMMARY_TOTAL_AMOUNT_OF_DEPOSITS_current_vs_6_months_ago']
        
        # don't need to include target variable for scoring
        cols_used_for_training.remove(target_attribute)

        # if a column does not exist in scoring but is in training, add the column to scoring dataset
        for col in cols_used_for_training:
            if col not in list(prepped_data.columns):
                prepped_data[col] = 0

        # if a column exists in scoring but not in training, delete it from scoring dataset
        for col in list(prepped_data.columns):
            if col not in cols_used_for_training:
                prepped_data.drop(col, axis=1, inplace=True)

        # make sure order of scoring columns is same as training dataset
        prepped_data = prepped_data[cols_used_for_training]
        
        prepped_data = prepped_data.drop(to_drop_corr, axis=1)
        
        return prepped_data
        
    def score(payload):
        
        import json
        try:
            sc_end_date = payload['input_data'][0]['values'][0][1]
        except:
            sc_end_date="2018-09-30"
            
        cust_id = payload['input_data'][0]['values'][0][0]
        
        prepped_data = prep(cust_id, sc_end_date)
        
        if prepped_data is None:
            return {"predictions" : [{'values' : 'Data prep filtered out customer data. Unable to score.'}]}
        else:

            scoring_payload = {"input_data":  [{ "values" : prepped_data.values.tolist()}]}
            
            response_scoring = client.deployments.score(parms['model_deployment_id'], scoring_payload)
            result=  response_scoring
            print(result['predictions'][0]['values'][0][1][1])
            result["Probability_of_Attrition"]=str(round(result['predictions'][0]['values'][0][1][1]*100,2))+"%"
            return {"predictions" : [{'values' : result}]}

    return score

### Deploy the Function

The user can specify the name of the function and deployment in the code below. 

In [26]:
# store the function and deploy it 
function_name = 'Customer Attrition Prediction Scoring Function'
function_deployment_name = 'Customer-Attrition-Prediction-Scoring-Function-Deployment'

We use tags, input data schemas, output data schemas and software specifications in the metadata to store the function. Input data schemas provides an easy option to input data to score in the deployment space. Example to create a metatadata to store the function can be viewed using `client.repository.FunctionMetaNames.get_example_values()`, similarly example to create a metatadata to deploy the function can be viewed using `client.deployments.ConfigurationMetaNames.get_example_values()` <br>
The Software Specification refers to the runtime used in the Notebook, WML training and WML deployment. We use the software specification `runtime-22.2-py3.10` to store the function. We get the ID of the software specification and include it in the metadata when storing the function. Available Software specifications can be retrieved using `client.software_specifications.list()`.


In [27]:

software_spec_id = client.software_specifications.get_id_by_name("runtime-22.2-py3.10")

In [28]:
# add the metadata for the function and deployment    
meta_data = {
    client.repository.FunctionMetaNames.NAME : function_name,
    client.repository.FunctionMetaNames.TAGS : ['attrition_scoring_pipeline_function_tag'],
    client.repository.FunctionMetaNames.INPUT_DATA_SCHEMAS:[{'id': '1','type': 'struct','fields': [{'name': 'CUSTOMER ID', 'type': 'int'},{'name': 'sc_end_date', 'type': 'date'}]}],
    client.repository.FunctionMetaNames.OUTPUT_DATA_SCHEMAS: [{'id': '1','type': 'struct','fields': [{'name': 'Probability_of_Attrition','type': 'double'}]}],
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: software_spec_id

}

function_details = client.repository.store_function(meta_props=meta_data, function=scoring_pipeline)

function_id = function_details["metadata"]["id"]

meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: function_deployment_name,
    client.deployments.ConfigurationMetaNames.TAGS : ['attrition_scoring_pipeline_function_deployment_tag'],
    client.deployments.ConfigurationMetaNames.DESCRIPTION:"Customer Attrition Scoring Function which will take raw data for scoring, prep it into the format required for the model and score it to return attrition probability of the customer.",
    client.deployments.ConfigurationMetaNames.SERVING_NAME: model_name.lower().replace(' ','')[:30]+''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(6))
}

# deploy the function
function_deployment_details = client.deployments.create(artifact_uid=function_id, meta_props=meta_props)




#######################################################################################

Synchronous deployment creation for uid: 'afb7a306-79db-4a2c-9268-9baf5d7b895f' started

#######################################################################################


initializing........
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='590eb0fd-1dd9-4752-808b-b559a1db19b4'
------------------------------------------------------------------------------------------------




### Score New Data

Get the guid of the deployed function, create the payload and use the python client to score the data. The deployed function returns the classification prediction along with the probabilities. 

The payload contains two values. The first is the effective date for scoring. This is the date that the prediction is computed. The scoring observation window and forecast horizon are calculated from this date. The second value contains the ID of the customer who we would like to make the prediction for. 

In [29]:
scoring_deployment_id = client.deployments.get_uid(function_deployment_details)
client.deployments.get_details(scoring_deployment_id)

{'entity': {'asset': {'id': 'afb7a306-79db-4a2c-9268-9baf5d7b895f'},
  'custom': {},
  'deployed_asset_type': 'function',
  'description': 'Customer Attrition Scoring Function which will take raw data for scoring, prep it into the format required for the model and score it to return attrition probability of the customer.',
  'hardware_spec': {'id': 'b128f957-581d-46d0-95b6-8af5cd5be580',
   'name': 'XXS',
   'num_nodes': 1},
  'name': 'Customer-Attrition-Prediction-Scoring-Function-Deployment',
  'online': {'parameters': {'serving_name': 'customerattritionpredictionmodid2k1t'}},
  'space_id': 'becc7261-e83a-462d-b515-934576db4e11',
  'status': {'online_url': {'url': 'https://internal-nginx-svc.wkc.svc.cluster.local:12443/ml/v4/deployments/590eb0fd-1dd9-4752-808b-b559a1db19b4/predictions'},
   'serving_urls': ['https://internal-nginx-svc.wkc.svc.cluster.local:12443/ml/v4/deployments/590eb0fd-1dd9-4752-808b-b559a1db19b4/predictions',
    'https://internal-nginx-svc.wkc.svc.cluster.local:

In [30]:
cust_id = 1008

payload = [{"fields":["CUSTOMER_ID","sc_end_date"],'values' : [[cust_id,"2018-09-30"]]}]

payload_metadata = {client.deployments.ScoringMetaNames.INPUT_DATA: payload}
# score
funct_output = client.deployments.score(scoring_deployment_id, payload_metadata)
funct_output

{'predictions': [{'values': {'predictions': [{'fields': ['prediction',
       'probability'],
      'values': [[0, [0.9207920945509181, 0.0792079054490819]]]}],
    'Probability_of_Attrition': '7.92%'}}]}

**The R Shiny Dashboard invokes this scoring pipeline for visualizing the results.***

# Deploy Shiny App

In this section we will complete the steps to deploy a Shiny Dashboard in Cloud Pak for Data. The app can be deployed in a similar way to models and functions, using the [ibm-watson-machine-learning](http://ibm-wml-api-pyclient.mybluemix.net) package.

All of the files associated with the dashboard are contained in a zip file which is stored in data assets. If the user would like to make changes to the dashboard, they can download the zip from data assets and upload it in the RStudio IDE. 

In [31]:
r_shiny_deployment_name='Customer-Attrition-Prediction-Shiny-App'

### Store the App

Create the associated metadata and store the dashboard zip file in the deployment space. 

In [32]:
rshiny_spec_uid=client.software_specifications.get_id_by_name("rstudio_r4.2")
# Meta_props to store assets in space 
meta_props = {
    client.shiny.ConfigurationMetaNames.NAME: "Customer Attrition Prediction Shiny Assets",
    client.shiny.ConfigurationMetaNames.DESCRIPTION: 'Store shiny assets in deployment space', # optional
    client.shiny.ConfigurationMetaNames.SOFTWARE_SPEC_UID:rshiny_spec_uid
}
app_details = client.shiny.store(meta_props, '/project_data/data_asset/customer-attrition-prediction-analytics-dashboard.zip')

Creating Shiny asset...
SUCCESS


### Deploy the App

Create the metadata for the Shiny deployment by providing  name, description, R-Shiny options and Hardware specifications. R-Shiny configuration provides options on whom you want to share the dashboard with, they are: 
<br>
* Anyone with the link 
* Authenticated users 
* Collaborators in this deployment space

In [ ]:
# Deployment metadata.
deployment_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: r_shiny_deployment_name,
    client.deployments.ConfigurationMetaNames.DESCRIPTION: 'Deploy Customer Attrition Prediction dashboard',
    client.deployments.ConfigurationMetaNames.R_SHINY: { 'authentication': 'anyone_with_url' },
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S', 'num_nodes': 1},
    client.deployments.ConfigurationMetaNames.SERVING_NAME:r_shiny_deployment_name.lower().replace('-','')[:30]+''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(6))
}

# Create the deployment.
app_uid = client.shiny.get_uid(app_details)
rshiny_deployment = client.deployments.create(app_uid, deployment_meta_props)



#######################################################################################

Synchronous deployment creation for uid: '2a78f701-8b56-4a5e-b0dc-175d8cf73e39' started

#######################################################################################


initializing......

### Launch Shiny App
Now that the dashboard is deployed, it can be accessed through the web browser. The app URL can be found by navigating to the deployed app in the deployment space. 

Open the Navigation Menu, select **Deployments -> Spaces -> Customer Attrition Space -> Deployments -> Customer-Attrition-Prediction-Shiny-App** to find the dashboard URL.

Alternatively, the path for the app URL can be found from the deployment metadata created in the previous cell. This path should be appended to the user's Cloud Pak for Data hostname to get the complete app URL. To get the path, run the cell below:

In [ ]:
print("{HOSTNAME}"+"/ml/v4/deployments/"+rshiny_deployment['metadata']['id'] + '/r_shiny')

<hr>

**Sample Materials, provided under license.</a> <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2019, 2023. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**